In [ ]:
## Import dependencies 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
%matplotlib inline

import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import plotly.figure_factory as ff
import plotly.offline as offline

import os
import sys
import pathlib
import glob
import gc
gc.enable()
import sys
import re
import math 
import random
import time 
import datetime as dt
from tqdm import tqdm 
from pprint import pprint

import warnings
warnings.filterwarnings('ignore')

import torch
from torch import nn
import torch.nn.functional as F
import torchvision
from torchvision import transforms
import torchvision.transforms.functional as TF
from torchvision.models import resnet18
#!pip install torchinfo -q --user
#from torchinfo import summary

from sklearn.model_selection import KFold

from PIL import Image

print('import done!')

In [ ]:
!mkdir -p /tmp/pip/cache/
!cp ../input/segmentationmodelspytorch-202208/torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl /kaggle/working/
#!pip install --no-index --find-links /tmp/pip/cache/ torch-1.11.0-cp37-cp37m-manylinux1_x86_64.whl

!cp ../input/segmentationmodelspytorch-202208/torchvision-0.12.0-cp37-cp37m-manylinux1_x86_64.whl /kaggle/working/
#!pip install --no-index --find-links /tmp/pip/cache/ torchvision-0.12.0-cp37-cp37m-manylinux1_x86_64.whl

#!cp ../input/segmentationmodelspytorch-202208/Pillow-9.1.1-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/certifi-2022.5.18.1-py3-none-any.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/charset_normalizer-2.0.12-py3-none-any.whl /kaggle/working/
##!cp ../input/segmentationmodelspytorch-202208/idna-3.3-py3-none-any.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/munch-2.5.0-py2.py3-none-any.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/numpy-1.21.6-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/requests-2.27.1-py2.py3-none-any.whl /kaggle/working/
!cp ../input/segmentationmodelspytorch-202208/segmentation_models_pytorch-0.2.1-py3-none-any.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/six-1.16.0-py2.py3-none-any.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/tqdm-4.64.0-py2.py3-none-any.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/typing_extensions-4.2.0-py3-none-any.whl /kaggle/working/
#!cp ../input/segmentationmodelspytorch-202208/urllib3-1.26.9-py2.py3-none-any.whl /kaggle/working/
!cp -rp ../input/segmentationmodelspytorch-202208/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3 /kaggle/working/efficientnet_pytorch-0.6.3.tar.gz
!cp -rp ../input/segmentationmodelspytorch-202208/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4 /kaggle/working/pretrainedmodels-0.7.4.tar.gz
!cp ../input/segmentationmodelspytorch-202208/timm-0.4.12-py3-none-any.whl /kaggle/working/

In [ ]:
!cp -rp ../input/segmentationmodelspytorch-202208/efficientnet_pytorch-0.6.3/efficientnet_pytorch-0.6.3 /kaggle/working/efficientnet_pytorch-0.6.3.tar.gz
!cp -rp ../input/segmentationmodelspytorch-202208/pretrainedmodels-0.7.4/pretrainedmodels-0.7.4 /kaggle/working/pretrainedmodels-0.7.4.tar.gz

In [ ]:
#tar.gzの中にもう一つディレクトリがあって、そのディレクトリにsetup.pyがあると動く
#efficientnet_pytorch-0.6.3.tar.gz/efficientnet_pytorch-0.6.3/setup.py
!pip install --no-index --find-links /tmp/pip/cache/ efficientnet_pytorch-0.6.3.tar.gz/efficientnet_pytorch-0.6.3
!pip install --no-index --find-links /tmp/pip/cache/ pretrainedmodels-0.7.4.tar.gz/pretrainedmodels-0.7.4
!pip install --no-index --find-links /tmp/pip/cache/ timm-0.4.12-py3-none-any.whl
!pip install --no-index --find-links /tmp/pip/cache/ segmentation_models_pytorch-0.2.1-py3-none-any.whl

In [ ]:
#resnetをオフラインで利用するための読み取り元のディレクトリを作成
!mkdir -p /root/.cache/torch/hub/checkpoints/
!cp ../input/segmentationmodelspytorch-202208/resnet34-333f7ec4.pth /root/.cache/torch/hub/checkpoints/

In [ ]:
import segmentation_models_pytorch as smp

In [ ]:
model = smp.DeepLabV3(encoder_name='resnet34',
                      encoder_depth=5,
                      encoder_weights='imagenet',
                      in_channels=1,
                      classes=4,
                      aux_params=None)
model.cuda()
#最適化手法
LEARNING_RATE = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

## For the model training loop.
if torch.cuda.is_available():
    DEVICE = 'cuda'
else: DEVICE = 'cpu'

In [ ]:
submission_df = pd.read_csv('../input/uw-madison-gi-tract-image-segmentation/sample_submission.csv')

In [ ]:
class UWMadison2022Dataset(torch.utils.data.Dataset):
    def __init__(self, files, dataframe=None, input_shape=256,predicted = False):
        self.files = files
        self.df = dataframe
        self.input_shape = input_shape
        self.transforms = transforms.Compose([
            transforms.CenterCrop(self.input_shape),
            transforms.Normalize(mean=[(0.485+0.456+0.406)/3], std=[(0.229+0.224+0.225)/3]),
        ])
        self.predicted = predicted
        
    def __len__(self):
        return len(self.files)
    
    def __getitem__(self, idx):
        p_file = self.files[idx]
        #img = torchvision.io.read_image(p_file)
        img = np.array(Image.open(p_file))
        img_shape = torch.tensor(img.shape)
        img = transforms.functional.to_tensor(img) / 255.
        img = self.transforms(img)
        #img = torch.cat([img, img, img], dim=0)
        
        if self.df is not None:
            f_name = str(p_file).split('/')
            case_day_id = f_name[5]
            slice_id = f_name[7][:10]
            f_id = '_'.join([case_day_id, slice_id])
            labels_df = self.df.query('id == @f_id')
            #display(labels_df.head())
            
            label = torch.zeros([img_shape[0]*img_shape[1]])
            if self.predicted == False:
                for i, organ in enumerate(['large_bowel', 'small_bowel', 'stomach']):
                    seg_type = organ + "_segmentation"
                    segmentation = labels_df[seg_type]#.item()
                    if type(segmentation) is str:
                        segmentation = segmentation.split(' ')
                        for j in range(len(segmentation)//2):
                            start_idx = int(segmentation[j*2])
                            span = int(segmentation[j*2 + 1])
                            label[start_idx:(start_idx+span)] = (i+1)
            else:
                segmentation = labels_df["predicted"]#.item()
                if type(segmentation) is str:
                    segmentation = segmentation.split(' ')
                    for j in range(len(segmentation)//2):
                        start_idx = int(segmentation[j*2])
                        span = int(segmentation[j*2 + 1])
                        label[start_idx:(start_idx+span)] = (i+1)
            
            label = torch.reshape(label, (img_shape[0], img_shape[1]))#256, 256pixelに固定
            label = transforms.CenterCrop(self.input_shape)(label)
            label = torch.nn.functional.one_hot(label.to(torch.int64), num_classes=4)
            label = label.permute(2, 0, 1)
            return img, label, img_shape
        
        else: return img, img_shape

## Prediction

In [ ]:
test_images = glob.glob(os.path.join("../input/uw-madison-gi-tract-image-segmentation/test", "**", "*.png"), recursive=True)
#test_images

#if len(test_images) == 0:
    #submission_df = pd.read_csv("../input/uw-madison-gi-tract-image-segmentation/train.csv")[["id", "class"]].iloc[:100 * 3]
    #submission_df["predicted"] = ""
    #test_images = glob.glob(os.path.join("../input/uw-madison-gi-tract-image-segmentation/train",  "**", "*.png"), recursive = True)

    
id2img = {_.rsplit("/", 4)[2] + "_" + "_".join(_.rsplit("/", 4)[4].split("_")[:2]): _ for _ in test_images}
submission_df["file_name"] = submission_df.id.map(id2img)
submission_df["days"] = submission_df.id.apply(lambda x: "_".join(x.split("_")[:2]))
fname2index = {f + c: i for f, c, i in zip(submission_df.file_name, submission_df["class"], submission_df.index)}
#submission_df


In [ ]:
submission_ids = np.unique(submission_df.id.values.tolist())
submission_df_unique_ids = submission_df[submission_df.index %3 == 0]
#submission_df_unique_ids.head()
#display(submission_df.head())
submission_df.shape

In [ ]:
gc.collect()
checkpoint = torch.load("../input/uwmgi-leraning-model/checkpoint.pth")
model.load_state_dict(checkpoint["model"])
optimizer.load_state_dict(checkpoint["optimizer"])
        
model.eval()
predictions = []
#predict_files = submission_df.file_name.values.tolist()
predict_files = submission_df_unique_ids.file_name.values.tolist()#1idにつき１filenameのリスト

predict_ds = UWMadison2022Dataset(predict_files, submission_df.reset_index(drop=True), input_shape=256, predicted = True)
BATCH_SIZE = 32

print('------ predict_dl ------')
predict_dl = torch.utils.data.DataLoader(predict_ds, batch_size=BATCH_SIZE, shuffle=False, drop_last=False)
tmp = predict_dl.__iter__()
x, y, shape = tmp.next()
print(f"x : {x.shape}")
print(f"labels: {y.shape}")
print(f"img_shapes: {shape.shape}")
print(f"n_samples: {len(predict_ds)}")
print(f"n_batches: {len(tmp)}")
print()
del predict_ds
gc.collect()


with torch.no_grad():
    for batch in tqdm(predict_dl):
        x = batch[0].to(DEVICE)
        predict_pred = model(x)
        predict_pred = torch.argmax(predict_pred, dim=1)
        predict_pred = torch.nn.functional.one_hot(predict_pred, num_classes=4)
        predict_pred = torch.permute(predict_pred, dims=[0, 3, 1, 2])
        predict_pred = predict_pred[:, 1:, ...] ## We don't need background predictions.
        predict_pred = predict_pred.detach().cpu().numpy()
        predictions.append(predict_pred)
    
predictions = np.concatenate(predictions, axis=0)
predictions = predictions.reshape([-1, 256, 256])
print(predictions.shape)

In [ ]:
def rle_encode(img):
    pixels = img.flatten()
    pixels = np.concatenate([[0], pixels, [0]])
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 1
    runs[1::2] -= runs[::2]
    encodes = ' '.join(str(x) for x in runs)
    if encodes == '':
        encodes = np.nan
    return encodes

predictions_rle = []

for pred in predictions:
    pred_rle = rle_encode(pred)
    predictions_rle.append(pred_rle)
    
predictions_rle = np.concatenate([predictions_rle], axis=0)
print("length : predictions_rle : ", len(predictions_rle))
submission_df['predicted'] = predictions_rle

#submission_df.head()
#submission_df[100:150]

In [ ]:
#submission_df.to_csv("submission_full_columns.csv")
submission_df = submission_df[["id", "class", "predicted"]]

submission_df.to_csv("submission.csv", index = False)

In [ ]:
#submission_df.head()